# Sistemas de Recomendación basados en SVD

## SVD: Siempre venciendo desafios

### Matemática Numérica
### Facultad de Matemática y Computación

Autores:
- Daniel Abad Fundora C212
- Anabel Benítez González C211
- Raudel Alejandro Gómez Molina C211
- Alex Sierra Alcalá C211

# Sistemas de Recomendación

Los sistemas de recomendación son una herramienta que le ofrece a los usuarios una determinada ayuda a la hora de tomar decisiones, basado en la experiencia de otros usuarios. Un sistema de recomendacion se puede definir como aquel sistema que tiene como principal tarea seleccionar ciertos objetos de acuerdo a los requerimientos del usuario. Estos sistemas son de gran utilidad cuando la cantidad de inforación ofrecida al usuario es muy susperior a las capacidades de evaluación y exploración de este.

La creación de un sistema de recomendación cuenta con tres fases principales:
- Captura de las preferencias y los gustos e intereses del usuario.
- Extracción del conocimiento, aquí el sistema se encarga de interpretar la informacion que recopiló anteriormente para poder predecir los gustos y las preferencias del usuario.
- A partir del contenido procesado anteriormente el sistema se encarga de se seleccionar los ítems que podrían interesarle a un determinado usario.

Ahora debemos encontrar un mecanismo para procesar toda la información que recopilemos previamente de los usuarios, para ello debemos reducir la dimensionalidad de nuestra matriz de votaciones, ya que esta puede ser muy grande y dispersa, a este proceso lo llamareos \textit{reducción de la dimensinalidad}. 

Por lo que es necesario determinar una matriz que sea equivalente a la original y que sea mas concisa a la hora de brindar la información para realizar la recomendación. Esto permite hacer más eficiente el proceso pues solo tendremos que considerar las características principales en vez de analizar completamente toda nuestra extensa matriz original, además esto permite minimizar los problemas relacionados con la presencia de datos erróneos en nuestra recopilación.

# Descomposición en Valores Singulares (SVD)

$$ \underset{(n, d)}A \approx \underset{(n, n)}U \cdot \underset{(n, d)}\Sigma \cdot \underset{(d, d)} V^T  $$

Cualquier matriz de tamaño (n, d) se puede descomponer en producto de tres factores

* En *U* de tamaño (n, n) es una matriz ortogonal que contiene los vectores singulares izquierdos de *A*.
* En $\Sigma$ que es una matriz diagonal (n,d), cuyos valores son los valores singulares de la matriz *A* ordenados en valor decreciente
* En *V* que es una matriz transpuesta (d,d), cuyos valores son los vectores singulares derechos de *A*.

*Ortogonal significa que multiplicando la transpuesta por si misma, se obtiene la matriz identidad*

Con esto lo que se consigue es que podemos ir elminando vectores de las matrices con la información que no es fundamental, (limpiar los datos) y quedarnos con aquella información más determinante.

## Aplicación práctica

Lo que se hace con los motores de recomendación, es para una película que tu no has visto, teniendo en cuenta tus características y las de otros usuarios. Mediante SVD nos quedamos con los usuarios que son parecidos a ti, y vemos las peliculas que no has visto
## Preprocesamos los datos

###  Cargar Bibliotecas

In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
from pandas.io.parsers import read_csv
from surprise import SVDpp
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict

### Cargar Datos 

In [31]:
items = pd.read_csv("data/items.csv")
ratings = pd.read_csv("data/ratings.csv")
df_items = ratings.merge(items, on="itemId", how="left")
df_items_to_model = df_items[df_items.columns[:3]]

### Preprocesamiento

In [32]:
reader = Reader()
data = Dataset.load_from_df(df_items_to_model[df_items.columns[:3]], reader)
train, test = train_test_split(data, test_size=0.25)


### Entrenamiento y testing

In [33]:
svd = SVDpp()
svd.fit(train)
preds = svd.test(test)

### Evaluación

It's a good evaluations marks so let's train the model with the complete Dataset

In [34]:
accuracy.mae(preds)
accuracy.rmse(preds)

MAE:  0.6601
RMSE: 0.8629


0.86292415185597

### Entrenar todos los datos

In [35]:
trainfull = data.build_full_trainset()

svd = SVDpp()
svd.fit(trainfull)

svd.predict(uid=1, iid=1)

Prediction(uid=1, iid=1, r_ui=None, est=4.673516457865482, details={'was_impossible': False})

### Función de Recomendación

In [36]:
def recommend_system(userId, dataframe, algorithm, n_commends):
    """
with the parameters, returns back the top n recommends items.

Parameters
-----------

userId: the user ID of the person that we want recommendations

dataframe: the DataFrame of items.

algorithm: the algorith used to recommend items.

n_commends: the number of items recommended.


return
------

ID of items that a specific user will like.

    """
    item_ids = dataframe['itemId'].to_list()
    items_watched = dataframe[dataframe["userId"] == userId]["itemId"]
    items_no_watched = [item for item in item_ids if item not in items_watched]

    preds = [algorithm.predict(uid=userId, iid=movie) for movie in items_no_watched]
    commends_ratting = {pred[1]:pred[3] for pred in preds}
    order_dict = {k: v for k, v in sorted(commends_ratting.items(), key=lambda item: item[1])}

    top_predictions = list(order_dict.keys())[:n_commends]

    return dataframe[dataframe["itemId"].isin(top_predictions)][["title", "genres"]].drop_duplicates()

In [37]:
items_recommended = recommend_system(3, df_items, svd, 5)
print("ID of the items recommended:", items_recommended)

ID of the items recommended:                                                 title                  genres
634    Mighty Morphin Power Rangers: The Movie (1995)         Action|Children
1316                                  Godzilla (1998)  Action|Sci-Fi|Thriller
14369                              Spice World (1997)                  Comedy
25257      Flintstones in Viva Rock Vegas, The (2000)         Children|Comedy
27285                          Problem Child 2 (1991)                  Comedy


### Comprobar los resultados

In [38]:
def check_items_user(userId, dataframe, n):
    return dataframe[dataframe["userId"] ==userId].sort_values("rating", ascending=False)[:n]

In [39]:
print(f"Items user likes:", check_items_user(1, df_items, 20))

Items user likes:      userId  itemId  rating  timestamp  \
231       1    5060     5.0  964984002   
185       1    2872     5.0  964981680   
89        1    1291     5.0  964981909   
90        1    1298     5.0  964984086   
190       1    2948     5.0  964982191   
189       1    2947     5.0  964982176   
188       1    2944     5.0  964981872   
186       1    2899     5.0  964982703   
184       1    2858     5.0  964980868   
179       1    2700     5.0  964980985   
98        1    1517     5.0  964981107   
100       1    1573     5.0  964982290   
102       1    1587     5.0  964982346   
103       1    1617     5.0  964982951   
181       1    2761     5.0  964982703   
105       1    1625     5.0  964983504   
88        1    1282     5.0  964982703   
87        1    1278     5.0  964983414   
86        1    1275     5.0  964982290   
85        1    1270     5.0  964983705   

                                                 title  \
231                       M*A*S*H (a.k.a.